# 7 Solution Methods to Solve the Growth Model with Python

This notebook is part of a computational appendix that accompanies the paper

> MATLAB, Python, Julia: What to Choose in Economics?
> > Coleman, Lyon, Maliar, and Maliar (2017)

In order to run the codes in this notebook you will need to install and configure a few Python packages. We recommend downloading [Anaconda](https://www.continuum.io/downloads) and/or following the instructions on [quantecon.org](https://lectures.quantecon.org/py/getting_started.html). Once your Python installation is up and running, there are a few additional packages you will need in order to run the code here. To do this, you should remove the `# ` in front of the following commands and run the block below.

In [35]:
# ! pip install git+https://github.com/EconForge/interpolation.py.git
# ! pip install quantecon

In [36]:
import numpy as np
import scipy.linalg as la
import scipy.optimize as opt
import time
import quantecon as qe

from collections import namedtuple
from interpolation.complete_poly import (CompletePolynomial,
                                         n_complete, complete_polynomial,
                                         complete_polynomial_der,
                                         _complete_poly_impl,
                                         _complete_poly_impl_vec,
                                         _complete_poly_der_impl,
                                         _complete_poly_der_impl_vec)
from numba import jit, vectorize

## Model

This section gives a short description of the commonly used stochastic Neoclassical growth model.

There is a single infinitely-lived representative agent who consumes and saves using capital. The consumer discounts the future with factor $\beta$ and derives utility from only consumption. Additionally, saved capital will depreciate at $\delta$.

The consumer has access to a Cobb-Douglas technology which uses capital saved from the previous period to produce and is subject to stochastic productivity shocks.

Productivity shocks follow an AR(1) in logs.

The agent's problem can be written recursively using the following Bellman equation


$$
\begin{align}
  V(k_t, z_t) &= \max_{k_{t+1}} u(c_t) + \beta E \left[ V(k_{t+1}, z_{t+1}) \right] \\
  &\text{subject to } \\
  c_t &= z_t f(k_t) + (1 - \delta) k_t - k_{t+1} \\
  \log z_{t+1} &= \rho \log z_t + \sigma \varepsilon
\end{align}
$$


## Python Code

We begin by defining a `namedtuple` that contains the parameters of our model. This is useful to pass the parameters around to functions that are just-in-time (JIT) compiled by `numba`.

In [37]:
#
# Create a named tuple type that we can pass into the jitted functions
# so that we don't have to pass parameters one by one
#
Params = namedtuple("Params", ["A", "alpha", "beta", "delta", "gamma",
                               "rho", "sigma"])

@jit(nopython=True)
def param_unpack(params):
    "Unpack parameters from the Params type"
    out = (params.A, params.alpha, params.beta, params.delta,
           params.gamma, params.rho, params.sigma)

    return out

We will then define various helper functions to ensure that we [don't repeat ourselves](https://lectures.quantecon.org/py/writing_good_code.html#don-t-repeat-yourself) and that the inner functions can be JIT compiled.

In [38]:
#
# Helper functions to make sure things are jitted
#
@vectorize(nopython=True)
def u(c, gamma):
    "CRRA utility function"
    return -1e10 if c < 1e-10 else (c**(1-gamma) - 1.0)/(1-gamma)

@vectorize(nopython=True)
def du(c, gamma):
    "Derivative of CRRA utility function"
    return 1e10 if c < 1e-10 else c**(-gamma)

@vectorize(nopython=True)
def duinv(u, gamma):
    "Inverse of the derivative of the CRRA utility function"
    return u**(-1.0 / gamma)

@vectorize(nopython=True)
def f(k, z, A, alpha):
    "C-D production function"
    return A * z * k**alpha

@vectorize(nopython=True)
def df(k, z, A, alpha):
    "Derivative of C-D production function"
    return alpha*A*z*k**(alpha - 1.0)

@vectorize(nopython=True)
def expendables_t(k, z, A, alpha, delta):
    "Budget constraint"
    return (1-delta)*k + f(k, z, A, alpha)


This code block contains other functions that are useful for computing the policy using the envelope condition (to be discussed later), simulating, and computing Euler errors.

In [39]:
@jit(nopython=True)
def env_cond_kp(temp, params, degree, v_coeffs, kt, zt):
    # Unpack parameters
    A, alpha, beta, delta, gamma, rho, sigma = param_unpack(params)

    # Compute derivative of VF wrt k
    _complete_poly_der_impl_vec(np.array([kt, zt]), degree, 0, temp)

    c = duinv(np.dot(temp, v_coeffs) / (1.0-delta+df(kt, zt, A, alpha)), gamma)

    return expendables_t(kt, zt, A, alpha, delta) - c


@jit(nopython=True)
def jit_simulate_ncgm(params, degree, v_coeffs, T, nburn, shocks):
    "Simulates economy using envelope condition as policy rule"
    # Unpack parameters
    A, alpha, beta, delta, gamma, rho, sigma = param_unpack(params)

    # Allocate space for output
    ksim = np.empty(T+nburn)
    zsim = np.empty(T+nburn)
    ksim[0], zsim[0] = 1.0, 1.0

    # Allocate space for temporary vector to fill with complete polynomials
    temp = np.empty(n_complete(2, degree))

    # Simulate
    for t in range(1, T+nburn):
        # Evaluate policy for today given yesterdays state
        kp = env_cond_kp(temp, params, degree, v_coeffs, ksim[t-1], zsim[t-1])

        # Draw new z and update k using policy from above
        zsim[t] = zsim[t-1]**rho * np.exp(sigma*shocks[t])
        ksim[t] = kp

    return ksim[nburn:], zsim[nburn:]


@jit(nopython=True)
def jit_ee(params, degree, v_coeffs, nodes, weights, ks, zs):
    # Unpack parameters
    A, alpha, beta, delta, gamma, rho, sigma = param_unpack(params)

    # Allocate space for temporary vector to fill with complete polynomials
    temp = np.empty(n_complete(2, degree))
    T = ks.size
    Qn = weights.size

    # Allocate space to store euler errors
    ee = np.empty(T)

    # Iterate over all ks and zs
    for t in range(T):
        # Current states
        k, z = ks[t], zs[t]

        # Compute decision for kp and implied c
        k1 = env_cond_kp(temp, params, degree, v_coeffs, k, z)
        c = expendables_t(k, z, A, alpha, delta) - k1

        # Compute euler error for period t
        lhs = du(c, gamma)
        rhs = 0.0
        for i in range(Qn):
            # Get productivity tomorrow
            z1 = z**rho * np.exp(nodes[i])

            # Compute decision for kpp and implied c
            k2 = env_cond_kp(temp, params, degree, v_coeffs, k1, z1)
            c1 = expendables_t(k1, z1, A, alpha, delta) - k2
            rhs = rhs + weights[i]*du(c1, gamma)*(1-delta+df(k1, z1, A, alpha))

        ee[t] = np.abs(1.0 - beta*rhs/lhs)

    return ee


We also now define a class that contains

1. Parameters of the growth model
2. Grids used for approximating the solution
3. Nodes and weights used to approximate integration

This again helps us maintain all of the relevant information in one place and simplifies passing it to other functions.

In [40]:
class NeoclassicalGrowth(object):
    """
    The stochastic Neoclassical Growth model contains
    parameters which include

    * alpha: Capital share in output
    * beta: discount factor
    * delta: depreciation rate
    * gamma: risk aversion
    * rho: persistence of the log productivity level
    * sigma: standard deviation of shocks to log productivity
    """
    def __init__(self, alpha=0.36, beta=0.99, delta=0.02, gamma=2.0,
                 rho=0.95, sigma=0.01, kmin=0.9, kmax=1.1, nk=10,
                 zmin=0.9, zmax=1.1, nz=10, Qn=5):

        # Household parameters
        self.beta, self.gamma = beta, gamma

        # Firm/technology parameters
        self.alpha, self.delta, self.rho, self.sigma = alpha, delta, rho, sigma

        # Make A such that CE steady state k is roughly 1
        self.A = (1.0/beta - (1-delta)) / alpha

        # Create t grids
        self.kgrid = np.linspace(kmin, kmax, nk)
        self.zgrid = np.linspace(zmin, zmax, nz)
        self.grid = qe.gridtools.cartesian([self.kgrid, self.zgrid])
        k, z = self.grid[:, 0], self.grid[:, 1]

        # Create t+1 grids
        self.ns, self.Qn = nz*nk, Qn
        eps_nodes, weights = qe.quad.qnwnorm(Qn, 0.0, sigma**2)
        self.weights = weights
        self.z1 = z[:, None]**rho * np.exp(eps_nodes[None, :])

    def _unpack_params(self):
        out = (self.A, self.alpha, self.beta, self.delta,
               self.gamma, self.rho, self.sigma)
        return out

    def _unpack_grids(self):
        out = (self.kgrid, self.zgrid, self.grid, self.ztp1, self.weights)
        return out


## Solution Methods

In this notebook, we describe the following solution methods:

* Conventional Value Function Iteration
* Envelope Condition Value Function Iteration
* Envelope Condition Derivative Value Function Iteration
* Endogenous Grid Value Function Iteration
* Conventional Policy Function Iteration
* Envelope Condition Policy Function Iteration
* Euler Equation Method

Each of these solution methods will have a very similar structure that follows a few basic steps:

1. Guess a function (either value function or policy function).
2. Using this function, update our guess of both the value and policy functions.
3. Check whether the function we guessed and what it was updated to are similar enough. If so, proceed. If not, return to step 2 using the updated functions.
4. Output the policy and value functions.

In order to reduce the amount of repeated code and keep the exposition as clean as possible (the notebook is plenty long as is...), we will define a class for each solution method that inherit various properties from a general solution parent class. The parent class will contain methods which apply to all of the other solution methods such as a general solve method, computing expectations, simulating, etc... This implementation may feel strange at first if one isn't familiar with object oriented programming, but these concepts can be powerful when properly used.

In [41]:
class GeneralSolution:
    """
    This is a general solution method. We define this, so that we can
    sub-class it and define specific update methods for each particular
    solution method
    """
    def __init__(self, ncgm, degree, prev_sol=None):
        # Save model and approximation degree
        self.ncgm, self.degree = ncgm, degree

        # Unpack some info from ncgm
        A, alpha, beta, delta, gamma, rho, sigma = self._unpack_params()
        grid = self.ncgm.grid
        k = grid[:, 0]
        z = grid[:, 1]

        # Use parameter values from model to create a namedtuple with
        # parameters saved inside
        self.params = Params(A, alpha, beta, delta, gamma, rho, sigma)

        self.Phi = complete_polynomial(grid.T, degree).T
        self.dPhi = complete_polynomial_der(grid.T, degree, 0).T

        # Update to fill initial value and policy matrices
        # If we give it another solution type then use it to
        # generate values and policies
        if issubclass(type(prev_sol), GeneralSolution):
            oldPhi = complete_polynomial(ncgm.grid.T, prev_sol.degree).T
            self.VF = oldPhi @ prev_sol.v_coeffs
            self.KP = oldPhi @ prev_sol.k_coeffs
        # If we give it a tuple then assume it is (policy, value) pair
        elif type(prev_sol) is tuple:
            self.KP = prev_sol[0]
            self.VF = prev_sol[1]
        # Otherwise guess a constant value function and a policy
        # of roughly steady state
        else:
            # VF is 0 everywhere
            self.VF = np.zeros(ncgm.ns)

            # Roughly ss policy
            c_pol = f(k, z, A, alpha) * (A-delta)/A
            self.KP = expendables_t(k, z, A, alpha, delta) - c_pol

        # Coefficients based on guesses
        self.v_coeffs = la.lstsq(self.Phi, self.VF)[0]
        self.k_coeffs = la.lstsq(self.Phi, self.KP)[0]

    def _unpack_params(self):
        return self.ncgm._unpack_params()

    def build_VF(self):
        """
        Using the current coefficients, this builds the value function
        for all states
        """
        VF = self.Phi @ self.v_coeffs

        return VF

    def build_KP(self):
        """
        Using the current coefficients, this builds the value function
        for all states
        """
        KP = self.Phi @ self.k_coeffs

        return KP

    def update_v(self, new_v_coeffs, dampen=1.0):
        """
        Updates the coefficients for the value function
        """
        self.v_coeffs = (1-dampen)*self.v_coeffs + dampen*new_v_coeffs

        return None

    def update_k(self, new_k_coeffs, dampen=1.0):
        """
        Updates the coefficients for the policy function
        """
        self.k_coeffs = (1-dampen)*self.k_coeffs + dampen*new_k_coeffs

        return None

    def update(self):
        """
        Given the current state of everything in solution, update the
        value and policy coefficients
        """
        emsg = "The update method is implemented in solution specific classes"
        emsg += "\nand cannot be called from `GeneralSolution`"
        raise ValueError(emsg)

    def compute_coefficients(self, kp, VF):
        """
        Given a policy and value return corresponding coefficients
        """
        new_k_coeffs = la.lstsq(self.Phi, kp)[0]
        new_v_coeffs = la.lstsq(self.Phi, VF)[0]

        return new_k_coeffs, new_v_coeffs

    def compute_EV_scalar(self, istate, kp):
        # Unpack parameters
        A, alpha, beta, delta, gamma, rho, sigma = self._unpack_params()

        # All possible exogenous states tomorrow
        z1 = self.ncgm.z1[istate, :]
        phi = complete_polynomial(np.vstack([np.ones(self.ncgm.Qn)*kp, z1]),
                                  self.degree).T
        val = self.ncgm.weights@(phi@self.v_coeffs)

        return val

    def compute_dEV_scalar(self, istate, kp):
        # Unpack parameters
        A, alpha, beta, delta, gamma, rho, sigma = self._unpack_params()

        # All possible exogenous states tomorrow
        z1 = self.ncgm.z1[istate, :]
        phi = complete_polynomial_der(np.vstack([np.ones(self.ncgm.Qn)*kp, z1]),
                                      self.degree, 0).T
        val = self.ncgm.weights@(phi@self.v_coeffs)
        print(val)
        return val

    def compute_EV(self, kp=None):
        """
        Compute the expected value
        """
        # Unpack parameters
        A, alpha, beta, delta, gamma, rho, sigma = self._unpack_params()
        grid = self.ncgm.grid
        ns, Qn = self.ncgm.ns, self.ncgm.Qn

        # Use policy to compute kp and c
        if kp is None:
            kp = self.Phi @ self.k_coeffs

        # Evaluate E[V_{t+1}]
        Vtp1 = np.empty((Qn, grid.shape[0]))
        for iztp1 in range(Qn):
            grid_tp1 = np.vstack([kp, self.ncgm.z1[:, iztp1]])
            Phi_tp1 = complete_polynomial(grid_tp1, self.degree).T
            Vtp1[iztp1, :] = Phi_tp1 @ self.v_coeffs

        EV = self.ncgm.weights @ Vtp1

        return EV

    def compute_dEV(self, kp=None):
        """
        Compute the derivative of the expected value
        """
        # Unpack parameters
        A, alpha, beta, delta, gamma, rho, sigma = self._unpack_params()
        grid = self.ncgm.grid
        ns, Qn = self.ncgm.ns, self.ncgm.Qn

        # Use policy to compute kp and c
        if kp is None:
            kp = self.Phi @ self.k_coeffs

        # Evaluate E[V_{t+1}]
        dVtp1 = np.empty((Qn, grid.shape[0]))
        for iztp1 in range(Qn):
            grid_tp1 = np.vstack([kp, self.ncgm.z1[:, iztp1]])
            dPhi_tp1 = complete_polynomial_der(grid_tp1, self.degree, 0).T
            dVtp1[iztp1, :] = dPhi_tp1 @ self.v_coeffs

        dEV = self.ncgm.weights @ dVtp1

        return dEV

    def envelope_policy(self):
        """
        Applies the envelope condition to compute the policy for
        k_{t+1} at every point on the grid
        """
        # Unpack parameters
        A, alpha, beta, delta, gamma, rho, sigma = self._unpack_params()
        grid = self.ncgm.grid
        k, z = grid[:, 0], grid[:, 1]

        dV = self.dPhi@self.v_coeffs

        # Compute the consumption
        temp = dV / (1 - delta + df(k, z, A, alpha))
        c = duinv(temp, gamma)

        return expendables_t(k, z, A, alpha, delta) - c

    def compute_distance(self, kp, VF):
        """
        Computes distance between policy functions
        """
        return np.max(np.abs(1.0 - kp/self.KP))

    def solve(self, tol=1e-6, maxiter=2500, verbose=False, nskipprint=25):
        # Iterate until convergence
        dist, it = 10.0, 0
        while (dist>tol) and (it<maxiter):
            # Run solution specific update code
            kp, VF = self.update()
            
            if it == 10:
                print('kp')
                print(kp)
                print('VF')
                print(VF)

            # Compute new policy and value coeffs
            new_k_coeffs, new_v_coeffs = self.compute_coefficients(kp, VF)

            # Update distance and iterations
            dist = self.compute_distance(kp, VF)
            self.KP, self.VF = kp, VF
            it += 1
            if verbose and it%nskipprint == 0:
                print(it, dist)

            # Update all coefficients
            self.update_k(new_k_coeffs)
            self.update_v(new_v_coeffs)

        # After finishing iteration, iterate to convergence using policy
        if not isinstance(self, IterateOnPolicy):
            sol_iop = IterateOnPolicy(self.ncgm, self.degree, self)
            kp, VF = sol_iop.solve(tol=1e-10)

            # Save final versions of everything
            self.KP, self.VF = kp, VF
            new_k_coeffs, new_v_coeffs = sol_iop.compute_coefficients(kp, VF)
            self.update_k(new_k_coeffs)
            self.update_v(new_v_coeffs)

        return self.KP, self.VF

    def simulate(self, T=10000, nburn=200, shocks=None, seed=42):
        """
        Simulates the neoclassical growth model with policy function
        given by self.KP. Simulates for `T` periods and discarsd first
        nburn `observations`
        """
        if shocks is None:
            np.random.seed(seed)
            shocks = np.random.randn(T+nburn)

        return jit_simulate_ncgm(self.params, self.degree, self.v_coeffs,
                                 T, nburn, shocks)

    def ee_residuals(self, ksim=None, zsim=None, Qn=10, seed=42):
        """
        Computes the Euler Equation residuals of a simulated capital and
        productivity levels (ksim and zsim) and uses Qn nodes for computing
        the expectation
        """
        if (ksim is None) or (zsim is None):
            ksim, zsim = self.simulate(T=10000, nburn=200, seed=seed)

        nodes, weights = qe.quad.qnwnorm(Qn, 0.0, self.ncgm.sigma**2)
        ee = jit_ee(self.params, self.degree, self.v_coeffs,
                    nodes, weights, ksim, zsim)

        return np.log10(np.mean(ee)), np.log10(np.max(ee))


### Iterating to Convergence (given policy)

This isn't one of the methods described above, but it is used as an element of a few of our methods (and also as a way to get a first guess at the value function). This method takes an initial policy function, $\bar{k}(k_t, z_t)$, as given, and then, without changing the policy, iterates until the value function has converged.

Thus the "update section" of the algorithm in this instance would be:

* Leave policy function unchanged
* At each point of grid, $(k_t, z_t)$, compute $\hat{V}(k_t, z_t) = u(c(\bar{k}(k_t, z_t))) + \beta E \left[ V(\bar{k}(k_t, z_t), z_{t+1}) \right]$

We override two of the methods from `GeneralSolution`

* `compute_distance` because when we are iterating to convergence on the value function we want to check distnace using value function rather than policy function
* `compute_coefficients` because we don't need to update the policy functions coefficients.

The `update` method just repeatedly applies a particular policy function to update the value function.

In [42]:
class IterateOnPolicy(GeneralSolution):
    """
    Subclass of the general solution method. The update method for this
    class simply computes the fixed point of the value function given
    a specific policy
    """
    def compute_distance(self, kp, VF):
        """
        Computes distance between policy functions. When we are
        iterating on a specific policy, we would like to compute
        distances by the difference between VFs
        """
        return np.max(np.abs(1.0 - VF/self.VF))

    def compute_coefficients(self, kp, VF):
        """
        Given a policy and value return corresponding coefficients.
        When we are iterating on a specific policy, we don't want to
        update the policy coefficients.
        """
        new_v_coeffs = la.lstsq(self.Phi, VF)[0]

        return self.k_coeffs, new_v_coeffs

    def update(self):
        # Unpack parameters
        A, alpha, beta, delta, gamma, rho, sigma = self._unpack_params()
        grid = self.ncgm.grid

        # Get the policy and update value function
        c = expendables_t(grid[:, 0], grid[:, 1], A, alpha, delta) - self.KP

        # Update the value function
        VF = u(c, gamma) + beta*self.compute_EV(self.KP)

        return self.KP, VF


### Conventional Value Function Iteration

This is one of the first solution methods for macroeconomics a graduate student in economics typically learns.

In this solution method, one takes as given a value function, $V(k_t, z_t)$, and then solves for the optimal policy given the value function.

The update section takes the form:

* For each point, $(k_t, z_t)$, numerically solve for $c^*(k_t, z_t)$ to satisfy the first order condition $u'(c^*) = \beta E \left[ V_1((1 - \delta) k_t + z_t f(k_t) - c^*, z_{t+1}) \right]$
* Define $k^*(k_t, z_t) = (1 - \delta) k_t + z_t f(k_t) - c^*(k_t, z_t)$
* Update value function according to $\hat{V}(k_t, z_t) = u(c^*(k_t, z_t)) + \beta E \left[ V(k^*(k_t, z_t), z_{t+1}) \right]$

In [43]:
class VFI(GeneralSolution):
    """
    Updates the coefficients and value functions using the VFI
    method
    """
    def update(self):
        """
        Updates the coefficients and value functions using the VFI_ECM
        method
        """
        # Unpack parameters
        A, alpha, beta, delta, gamma, rho, sigma = self._unpack_params()
        grid = self.ncgm.grid
        n_state = grid.shape[0]

        # Get the policy and update it
        kp = np.empty(n_state)
        VF = np.empty(n_state)
        for i_s in range(n_state):
            # Pull out current vals
            k, z = grid[i_s, :]

            # Compute possible expendables
            expend_t = expendables_t(k, z, A, alpha, delta)

            # Negative of the objective function since we are minimizing
            _f = lambda kp: (du(expend_t-kp, gamma) -
                             beta*self.compute_dEV_scalar(i_s, kp))
            _kp = opt.brentq(_f, 0.25, expend_t - 1e-2, rtol=1e-12)

            kp[i_s] = _kp
            VF[i_s] = u(expend_t-_kp, gamma)+beta*self.compute_EV_scalar(i_s, _kp)

        return kp, VF


### Endogenous Grid Value Function Iteration

Method introduced by Chris Carroll. The key to this method is that the grid of points being used to approximate is over $(k_{t+1}, z_{t})$ instead of $(k_t, z_t)$. The insightful piece of this algorithm is that the transformation allows one to write a closed form for the consumption function, $c^*(k_{t+1}, z_t) = u'^{-1} \left( V_1(k_{t+1}, z_{t+1}) \right]$.

Then for a given $(k_{t+1}, z_{t})$ the update section would be

* Define $c^*(k_{t+1}, z_t) = u'^{-1} \left( \beta E \left[ V_1(k_{t+1}, z_{t+1}) \right] \right)$
* Find $k_t$ such that $k_t = (1 - \delta) k_t + z_t f(k_t) - k_{t+1}$
* Update value function according to $\hat{V}(k_t, z_t) = u(c^*(k_{t+1}, z_t)) + \beta E \left[ V(k_{t+1}, z_{t+1}) \right]$


In [44]:
class VFI_EGM(GeneralSolution):
    def update(self):
        """
        Updates the coefficients and value functions using the VFI_ECM
        method
        """
        # Unpack parameters
        A, alpha, beta, delta, gamma, rho, sigma = self._unpack_params()
        ns = self.ncgm.ns
        grid = self.ncgm.grid

        # Use the grid as capital tomorrow instead of capital today
        k1 = grid[:, 0]
        z = grid[:, 1]
        dEV = self.compute_dEV(kp=k1)
        c = duinv(beta*dEV, gamma)

        # Now find corresponding capital today such that kp is optimal
        kt = np.empty(ns)
        for i in range(ns):
            ct = c[i]
            ktp1 = k1[i]
            zt = z[i]
            obj = lambda k: expendables_t(k, zt, A, alpha, delta) - ct - ktp1
            kt[i] = opt.bisect(obj, 0.25, 2.0)

        # Compute value today
        EV = self.compute_EV(kp=k1)
        V = u(c, gamma) + beta*EV

        # Get implied coefficients and evaluate kp and VF
        phi = complete_polynomial(np.vstack([kt, z]), self.degree).T
        temp_k_coeffs = la.lstsq(phi, k1)[0]
        temp_v_coeffs = la.lstsq(phi, V)[0]
        kp = self.Phi@temp_k_coeffs
        VF = self.Phi@temp_v_coeffs

        return kp, VF


### Envelope Condition Value Function Iteration

Very similar to the previous method. The insight of this algorithm is that since we are already approximating the value function and can evaluate its derivative, we can skip the numerical optimization piece of the update method and compute directly the policy using the envelope condition (hence the name).

The envelope condition says:

$$c^*(k_t, z_t) = u'^{-1} \left( \frac{\partial V(k_t, z_t)}{\partial k_t} (1 - \delta + r)^{-1} \right)$$

so

$$k^*(k_t, z_t) = z_t f(k_t) + (1-\delta)k_t - c^*(k_t, z_t)$$

We defined functions which get the policy using the envelope condition in the helper function section and inside the `GeneralSolution` class

The update method is then very similar to other value iteration style methods, but avoids the numerical solver.

* For each point, $(k_t, z_t)$ get $c^*(k_t, z_t)$ from the envelope condition
* Define $k^*(k_t, z_t) = (1 - \delta) k_t + z_t f(k_t) - c^*(k_t, z_t)$
* Update value function according to $\hat{V}(k_t, z_t) = u(c^*(k_t, z_t)) + \beta E \left[ V(k^*(k_t, z_t), z_{t+1}) \right]$

In [45]:
class VFI_ECM(GeneralSolution):
    """
    This class implements the Envelope Condition Method for Value
    Function Iteration.
    """
    def update(self):
        """
        Updates the coefficients and value functions using the VFI_ECM
        method
        """
        # Unpack parameters
        A, alpha, beta, delta, gamma, rho, sigma = self._unpack_params()
        grid = self.ncgm.grid

        # Get the policy and update it
        kp = self.envelope_policy()
        c = expendables_t(grid[:, 0], grid[:, 1], A, alpha, delta) - kp

        # Update the value function
        VF = u(c, gamma) + beta*self.compute_EV(kp)

        return kp, VF


### Conventional Policy Function Iteration

Policy function iteration is different than value function iteration in that it starts with a policy function, then updates the value function, and finally finds the new optimal policy function. Given a policy $c(k_t, z_t)$ and for each pair $(k_t, z_t)$

* Define $k(k_t, z_t) = (1 - \delta) k_t + z_t f(k_t) - c(k_t, z_t)$
* Find fixed point of $V(k_t, z_t) = u(c(k_t, z_t)) + \beta E \left[ V(k(k_t, z_t), z_t) \right]$ (Iterate to convergence given policy)
* Given $V(k_t, z_t)$, numerically solve for new policy $c^*(k_t, z_t)$ -- Stop when $c(k_t, z_t) \approx c^*(k_t, z_t)$

In [46]:
class PFI(GeneralSolution):
    def update(self):
        """
        Updates the coefficients and value functions using the PFI
        method
        """
        # Unpack parameters
        A, alpha, beta, delta, gamma, rho, sigma = self._unpack_params()
        ns = self.ncgm.ns
        grid = self.ncgm.grid

        # Update policy and use new policy to iterate to convergence
        kp = np.empty(ns)
        for i_s in range(ns):
            # Pull out current vals
            k, z = grid[i_s, :]

            # Compute possible expendables
            expend_t = expendables_t(k, z, A, alpha, delta)

            # Negative of the objective function since we are minimizing
            _f = lambda kp: (du(expend_t-kp, gamma) -
                             beta*self.compute_dEV_scalar(i_s, kp))
            _kp = opt.brentq(_f, 0.25, expend_t - 1e-2, rtol=1e-12)
            kp[i_s] = _kp

        sol_itp = IterateOnPolicy(self.ncgm, self.degree, (kp, self.VF))
        _, VF = sol_itp.solve(maxiter=5000)

        return kp, VF


### Envelope Condition Policy Function Iteration

Similar to policy function iteration, but, rather than numerically solve for new policies, it uses the envelope condition to directly compute them. Given a starting policy $c(k_t, z_t)$ and for each pair $(k_t, z_t)$

* Define $k(k_t, z_t) = (1 - \delta) k_t + z_t f(k_t) - c(k_t, z_t)$
* Find fixed point of $V(k_t, z_t) = u(c(k_t, z_t)) + \beta E \left[ V(k(k_t, z_t), z_t) \right]$ (Iterate to convergence given policy)
* Given $V(k_t, z_t)$ find $c^*(k_t, z_t)$ using envelope condition -- Stop when $c(k_t, z_t) \approx c^*(k_t, z_t)$

In [47]:
class PFI_ECM(GeneralSolution):
    """
    Updates the coefficients and value functions using the PFI_ECM
    method
    """
    def update(self):
        """
        Updates the coefficients and value functions using the PFI_ECM
        method
        """
        # Unpack parameters
        A, alpha, beta, delta, gamma, rho, sigma = self._unpack_params()
        ns = self.ncgm.ns
        grid = self.ncgm.grid

        # Update policy and use new policy to iterate to convergence
        kp = self.envelope_policy()
        sol_itp = IterateOnPolicy(self.ncgm, self.degree, (kp, self.VF))
        _, VF = sol_itp.solve(maxiter=5000)

        return kp, VF


### Euler Equation Method

Euler equation methods operate directly on the Euler equation: $u'(c_t) = \beta E \left[ u'(c_{t+1}) (1 - \delta + z_t f'(k_t)) \right]$.

Given an initial policy $c(k_t, z_t)$ for each grid point $(k_t, z_t)$

* Find $k(k_t, z_t) = (1-\delta)k_t + z_t f(k_t) - c(k_t, z_t)$
* Let $c_{t+1} = c(k(k_t, z_t), z_t)$
* Numerically solve for a $c^*$ that satisfies the Euler equation i.e. $u'(c^*) = \beta E \left[ u'(c_{t+1}) (1 - \delta + z_t f'(k_t)) \right]$
* Stop when $c^*(k_t, z_t) \approx c(k_t, z_t)$

In [48]:
class EulEq(GeneralSolution):
    def update(self):
        # Unpack parameters
        A, alpha, beta, delta, gamma, rho, sigma = self._unpack_params()
        ns, Qn = self.ncgm.ns, self.ncgm.Qn
        ncgm, grid = self.ncgm, self.ncgm.grid
        k, z = grid[:, 0], grid[:, 1]

        # Evaluate RHS of EE
        temp = np.empty((n_complete(2, self.degree), ns))
        rhs_ee = 0.0
        for iz1 in range(Qn):
            # Build t+1 decisions
            zp = ncgm.z1[:, iz1]
            _complete_poly_impl(np.vstack([self.KP, zp]), self.degree, temp)

            # Compute k_{t+2} and the corresponding c_{t+1}
            kpp = self.k_coeffs@temp
            cp = expendables_t(self.KP, zp, A, alpha, delta) - kpp
            foo = beta*ncgm.weights[iz1]*(1.0-delta+df(self.KP, zp, A, alpha))
            foo *= du(cp, gamma)
            rhs_ee += foo

        # Determine c_t using euler equation
        c = duinv(rhs_ee, gamma)
        kp = expendables_t(k, z, A, alpha, delta) - c
        VF = u(c, gamma) + beta*self.compute_EV(kp=kp)

        return kp, VF


### Envelope Condition Derivative Value Function Iteration

This method uses the same insight of the "Envelope Condition Value Function Iteration," but, rather than iterate directly on the value function, it iterates on the derivative of the value function. The update steps are

* For each point, $(k_t, z_t)$ get $c^*(k_t, z_t)$ from the envelope condition (which only depends on the derivative of the value function!)
* Define $k^*(k_t, z_t) = (1 - \delta) k_t + z_t f(k_t) - c^*(k_t, z_t)$
* Update value function according to $\hat{V}_1(k_t, z_t) = \beta (1 - \delta + z_t f'(k_t)) E \left[ V_1(k^*(k_t, z_t), z_{t+1}) \right]$

Once it has converged, you use the implied policy rule and iterate to convergence using the "iterate to convergence (given policy)" method.

Here we override the `compute_coefficients` method to ensure that we are using the derivative of basis matrices to update the coefficients.

In [49]:
class dVFI_ECM(GeneralSolution):
    def compute_coefficients(self, kp, dVF):
        """
        Given a policy and derivative of the value function
        return corresponding coefficients
        """
        new_k_coeffs = la.lstsq(self.Phi, kp)[0]
        new_v_coeffs = la.lstsq(self.dPhi, dVF)[0]

        return new_k_coeffs, new_v_coeffs

    def update(self):
        # Unpack parameters
        A, alpha, beta, delta, gamma, rho, sigma = self._unpack_params()
        grid = self.ncgm.grid

        # Get the policy and update it
        kp = self.envelope_policy()
        c = expendables_t(grid[:, 0], grid[:, 1], A, alpha, delta) - kp

        dEV = self.compute_dEV(kp=kp)
        dV = (1-delta+df(grid[:, 0], grid[:, 1], A, alpha))*beta*dEV

        return kp, dV


### Simulation and Euler Error Methods

We have already defined functions which simulate and compute Euler errors for these solution methods.

## A Horse Race

We can now run a horse race to compare the methods in terms of both accuracy and speed.

In [50]:
ncgm = NeoclassicalGrowth()

# First guess
vp = IterateOnPolicy(ncgm, 2)
vp.solve(tol=1e-9)

np.random.seed(61089)
shocks = np.random.randn(10200)

# for sol_method in [VFI, VFI_ECM, VFI_EGM, PFI, PFI_ECM, dVFI_ECM, EulEq]:
for sol_method in [VFI ]:

    
    # Set prev sol as iterate on policy
    new_sol = vp
    print("Solution Method: {}\n".format(sol_method))

    for d in range(2, 6):
        new_sol = sol_method(ncgm, d, new_sol)
        ts = time.time()
        new_sol.solve(tol=1e-9, verbose=False, nskipprint=25)
        time_took = time.time() - ts
        print("\tDegree {} took {}\n".format(d, time_took))

        # Compute Euler Errors
        ks, zs = new_sol.simulate(10000, 200, shocks=shocks)
        mean_ee, max_ee = new_sol.ee_residuals(ks, zs, Qn=10)
        print("\t\tMean and Max EE are {} & {}\n".format(mean_ee, max_ee))
        new_sol = new_sol

<ipython-input-42-7d2ee4374b26>:13: RuntimeWarning: divide by zero encountered in true_divide
  return np.max(np.abs(1.0 - VF/self.VF))


kp
[0.89933005 0.89975795 0.90018585 0.90061376 0.90104166 0.90146956
 0.90189746 0.90232537 0.90275327 0.90318117 0.92126067 0.92169235
 0.92212402 0.9225557  0.92298738 0.92341905 0.92385073 0.92428241
 0.92471408 0.92514576 0.94318895 0.94362435 0.94405974 0.94449513
 0.94493052 0.94536592 0.94580131 0.9462367  0.94667209 0.94710749
 0.96511499 0.96555404 0.96599309 0.96643215 0.9668712  0.96731025
 0.96774931 0.96818836 0.96862741 0.96906646 0.98703885 0.98748151
 0.98792417 0.98836683 0.98880949 0.98925216 0.98969482 0.99013748
 0.99058014 0.9910228  1.00896063 1.00940685 1.00985307 1.01029928
 1.0107455  1.01119171 1.01163793 1.01208415 1.01253036 1.01297658
 1.0308804  1.03133013 1.03177985 1.03222957 1.03267929 1.03312901
 1.03357874 1.03402846 1.03447818 1.0349279  1.05279823 1.05325141
 1.05370459 1.05415777 1.05461095 1.05506413 1.05551731 1.05597049
 1.05642367 1.05687685 1.07471419 1.07517078 1.07562737 1.07608396
 1.07654055 1.07699714 1.07745374 1.07791033 1.07836692 1.0

454.2038541848753
278.1750048740556
446.45741572075644
362.3162102974061
320.24560758573085
299.21030622989326
288.69265555197444
292.22562215972965
290.88685108236086
290.57520385764735
290.58522049129976
290.58493263431967
290.58493236005
290.584932360417
451.4978816443719
275.01557630122255
443.77813711905895
359.3968567101408
317.20621650568165
296.1108964034521
285.56323635233736
289.1741906954444
287.8186245059032
287.4949183625398
287.50564864742427
287.50533078203125
287.50533045972253
287.5053304600897
448.79516767163
271.859406296151
441.1022977615022
356.4808520288266
314.1701291624888
293.01476772931994
282.43708701273545
286.12719804482344
284.755122400026
284.4190289023406
284.4305132694791
284.43016276084364
284.4301623829836
284.43016238335093
446.0956317931721
268.7064143853635
438.4298181058753
353.5681162456194
311.13726531549145
289.92183985042755
279.3141271178955
283.084574975692
281.6962959209785
281.34748459567146
281.35976487809273
281.3593789184092
281.3593784

274.1995791899154
274.2024020265438
274.20235866864044
274.2023586571831
274.20235865680854
451.2643665454955
258.14889342988073
442.8290364776209
350.48896495375084
304.3189291918158
281.23391131084827
269.6914023703645
272.422793287191
271.2355657281234
271.0566061922106
271.0598333522201
271.05978058621906
271.05978057039295
271.0597805700183
448.50065516100017
254.9209139476744
440.0972609357891
347.5090874417317
301.21500069470306
278.0679573211887
266.4944356344315
269.32939524869835
268.11007486549994
267.9183422188058
267.9220174318327
267.9219535911726
267.92195356952533
267.9219535691505
445.7401935970343
251.69618428599736
437.3689206260462
344.53255245602185
298.11436837100956
274.90527632850353
263.3007303072504
266.24073605778517
264.9897950554433
264.78474200382584
264.78891214247324
264.78883533280697
264.788835303473
264.7888353038481
442.98290344074996
248.47462603200182
434.64393815052364
341.5592820912627
295.0169540616323
271.74579004681704
260.11020803940943
263.1

239.4576125542279
238.16233654955772
237.96116203857105
237.96494622391774
237.96488140684994
237.96488138558843
237.96488138520775
428.5669737370878
220.77570924175703
419.95659496590156
320.36615210382934
270.5709306727932
245.67331995727508
233.22451459951606
236.35707579532516
235.0208796638841
234.80241212724744
234.80683316642094
234.8067517526009
234.80675172166863
234.80675172204943
453.85395192634417
244.65374698423642
444.6728403214797
344.66329365285804
294.6585203185472
269.6561336513918
257.1549403178142
258.79538616084204
257.9871924241784
257.9250479650337
257.9254314185545
257.92542924776296
257.92542924738103
451.03142363271706
241.35665086691043
441.88718965299904
341.6219202599547
291.4892855634326
266.4229682151715
253.8898095410409
255.66218186281054
254.79883108990325
254.72667687849608
254.7271895811991
254.72718623983644
254.7271862394543
448.212294264824
238.06295367531845
439.1051252807156
338.584039478017
288.32349657666776
263.1932251259931
250.6280894006558

241.47767927255754
242.09672671196708
241.76324279237213
241.75500354225863
241.7547953317477
241.75479533592195
241.75479533553258
450.79979197399706
224.63328562546383
440.95289227111545
332.7930889482897
278.71318728687675
251.6732364561703
238.1532610408171
238.94016244007832
238.52149844153763
238.5083569894859
238.5079324127584
238.50793242997463
238.50793242958508
447.93034128854407
221.27942283457475
438.1256857875182
329.70255431104647
275.4909885728106
248.38520570369263
234.8323142691337
235.78921204079927
235.28654762611865
235.26737131623193
235.26661345589702
235.26661351029753
235.26661350990776
445.0642647614052
217.9289342019998
435.3020446454625
326.61548942373116
272.2722118128654
245.10057300743262
231.5147536047162
232.64380691234834
232.0583928572516
232.0320563692544
232.0308208099185
232.03082095330012
232.03082095291023
442.20148097966523
214.58173831482367
432.48188872361493
323.5318135192192
269.05677591702147
241.8192571159226
228.2004977153731
229.503882332

356.45788495551164
312.108041691283
289.9331200591687
278.8456592431115
281.7324340953879
280.5158469113777
280.3102458487732
280.3146097018803
280.31452605355196
280.31452601882586
280.31452601845706
450.3429486649085
264.3412531570187
442.2311624815268
353.2862078192727
308.8137304881457
286.5774918225822
275.4593724898005
278.44757951427425
277.2020095216457
276.98297256366226
276.9878856032153
276.98778614172267
276.9877860955827
276.9877860959517
447.38641646807787
260.927872474505
439.3085136697897
350.11819307214733
305.5230327733262
283.2254526239156
272.0766625492103
275.1677748894738
273.8937196120453
273.6607130036619
273.66622737139505
273.6661096602429
273.6661095994174
273.6661095990481
444.4333608278381
257.51796834858226
436.38953811443633
346.95375323150927
302.23586079004576
279.87691456931395
268.6974414589481
271.8929456786072
270.5909357951943
270.3434199859856
270.3495909922767
270.34945231046424
270.349452230884
270.34945223125345
441.48369786057145
254.111456895

240.67894634074491
435.39325651077763
338.0361014257613
289.35752388325307
265.01823511199905
252.84859072637198
255.16346786890216
254.09742060349691
253.9727794728621
253.97431077471214
253.97429359704975
253.97429359465485
253.97429359427787
441.14705205194315
237.22443943178212
432.4454557614627
334.83494759662244
286.0296935142023
261.6270664729922
249.42575295238714
251.87620032212783
250.76199858717774
250.62315542340014
250.62503845024202
250.62501514551678
250.62501514155474
250.62501514117764
438.1632971644841
233.7732872301554
429.50121081059183
331.6372490203736
282.7052681252645
258.23927767771
246.00628245393267
248.5941416877376
247.4327047799276
247.27878844439365
247.28108126593077
247.28105011662282
247.2810501102294
247.28105011060666
435.1828195979637
230.32541234946734
426.56044532490387
328.4429288371856
279.3841705933265
254.8547914713969
242.5901019104321
245.31722970174536
244.10952993160794
243.9396582403213
243.9424249220073
243.94238385308105
243.94238384299

233.09153580256185
219.77197853858308
222.0493020505943
220.97736578058766
220.8657405429429
220.8668753791409
220.86686476305027
220.8668647620309
220.86686476241624
425.76869424642155
202.97013829684147
416.6059267045313
309.7880325006864
256.37908539876395
229.67461184780274
216.32237507232205
218.78206849013162
217.64229205645114
217.51294882724153
217.5144562266761
217.51444007052075
217.51444006848035
217.51444006809487
452.9297047026724
228.83843907799576
443.1250721891963
335.9817556335961
282.4100973557959
255.6242682168958
242.23135364744576
242.79126010353764
242.48833038294163
242.48151762949513
242.4813612410503
242.48136124342733
242.48136124304068
449.8979972395465
225.32573412078352
440.1386139041565
332.73217401246995
279.02895406662674
252.17734409370513
238.75153910724435
239.48848381931396
239.09498704930598
239.08334429201682
239.08299032536902
239.08299033744294
239.08299033705617
446.8699405984647
221.81667998561542
437.15600527727645
329.48634263144595
275.65151

KeyboardInterrupt: 

In [54]:
np.array([[1,1][1,0],[0,1]])

<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
<ipython-input-54-81a6ad56d9b1>:1: SyntaxWarning: list indices must be integers or slices, not tuple; perhaps you missed a comma?
  np.array([[1,1][1,0],[0,1]])


TypeError: list indices must be integers or slices, not tuple

In [55]:
A = np.array([[1,1],[1,0],[0,1]])
x = np.array([3,2])
A.dot(x)

array([5, 3, 2])